In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import sklearn.model_selection as skmodel
import import_ipynb
import loaddataset

importing Jupyter notebook from loaddataset.ipynb


In [2]:
X_train, X_test, Y_train, Y_test = loaddataset.loaddata()

Shape of the data is:
    X_train: (1216, 50)
    Y_train: (1216, 8)
    X_test:  (304, 50)
    Y_test:  (304, 8)
    


In [3]:
training_epochs = 700
display_step = 100
batch_size = 30
n_hidden_1 = 90

n_input = X_train.shape[1]
n_classes = Y_train.shape[1]

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


In [4]:
def multilayer_perceptron(x, weights, biases, rate):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, rate)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [5]:

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

rate = tf.placeholder("float")

In [6]:
predictions = multilayer_perceptron(x, weights, biases, rate)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.007).minimize(cost)
warnings.filterwarnings('ignore')


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
saver = tf.train.Saver()
model_path = "trainedmodel/model.ckpt"

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(Y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                rate: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test, rate: 1.0}))      
    print("Optimization Finished!")
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)

Epoch: 0001 cost= 9.334566736
Epoch: 0101 cost= 0.420013301
Epoch: 0201 cost= 0.226101288
Epoch: 0301 cost= 0.167253949
Epoch: 0401 cost= 0.092508383
Epoch: 0501 cost= 0.096659646
Epoch: 0601 cost= 0.078789238
Accuracy: 0.7927632
Optimization Finished!
Model saved in file: trainedmodel/model.ckpt
